In [2]:
import pandas as pd
from pathlib import Path
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径和常量"""
    # --- 【请修改】输入和输出路径 ---
    # 输入文件夹：包含所有地市Excel文件的文件夹
    INPUT_DIR = Path(r'E:\A智网\分地市202508充电负荷数据\充电负荷数据')
    
    # 输出文件：最终汇总表的保存路径和文件名
    OUTPUT_FILE = Path(r'E:\A智网\分地市202508充电负荷数据\湖北省充电负荷数据.xlsx')

    # --- 文件读取参数 ---
    # 根据你的Excel文件格式，我们需要跳过前面的2行
    ROWS_TO_SKIP = 2

# ==============================================================================
# 2. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """主执行函数，完成数据读取、聚合和保存"""
    print("--- 开始执行充电负荷数据批量加总任务 ---")

    # 检查输入文件夹是否存在
    if not Config.INPUT_DIR.exists():
        print(f"[致命错误] 输入文件夹不存在: {Config.INPUT_DIR}")
        return

    # 1. 读取所有地市的Excel文件
    all_city_dataframes = []
    # 使用 .glob('*.xls*') 可以同时匹配 .xls 和 .xlsx 文件
    excel_files = list(Config.INPUT_DIR.glob('*.xls*'))

    if not excel_files:
        print(f"[致命错误] 在文件夹 {Config.INPUT_DIR} 中没有找到任何Excel文件。")
        return

    print(f"发现 {len(excel_files)} 个Excel文件，准备开始读取...")

    for file_path in excel_files:
        print(f"  -> 正在读取文件: {file_path.name}")
        try:
            # 使用 skiprows 参数跳过文件顶部的标题和空行
            df = pd.read_excel(file_path, skiprows=Config.ROWS_TO_SKIP)
            
            # 简单的有效性检查，确保DataFrame包含预期的数据
            if '日期' in df.columns and len(df.columns) > 90:
                all_city_dataframes.append(df)
            else:
                print(f"    [警告] 文件 {file_path.name} 的格式似乎不正确，已跳过。")

        except Exception as e:
            print(f"    [错误] 读取文件 {file_path.name} 时发生错误，已跳过。")
            print(traceback.format_exc())
            continue
    
    if not all_city_dataframes:
        print("\n[处理中断] 未能成功读取任何有效数据，请检查上述错误日志。")
        return

    # 2. 合并所有数据到一个大的DataFrame
    print("\n所有文件读取完毕，正在合并数据...")
    combined_df = pd.concat(all_city_dataframes, ignore_index=True)
    print("数据合并完成！")

    # 3. 数据清洗与转换
    print("正在进行数据清洗和类型转换...")
    
    # a. 转换“日期”列为标准的日期格式
    #    format='%Y%m%d' 告诉pandas如何解析像 20250801 这样的整数/字符串
    combined_df['日期'] = pd.to_datetime(combined_df['日期'], format='%Y%m%d')

    # b. 确定所有时间点列（从第4列到最后一列）
    time_point_columns = combined_df.columns[3:]

    # c. 将所有时间点列转换为数值类型，无法转换的将变成NaN
    #    然后用0填充所有NaN值，确保加总时不会出错
    combined_df[time_point_columns] = combined_df[time_point_columns].apply(pd.to_numeric, errors='coerce').fillna(0)
    print("数据清洗完成！")

    # 4. 核心聚合操作：按日期分组，并对所有时间点列求和
    print("正在按天对所有地市的负荷进行加总...")
    # groupby('日期') 会将所有相同日期的数据行分到一组
    # .sum() 会自动对该组内所有数值列（即我们的96个时间点）进行求和
    aggregated_df = combined_df.groupby('日期')[time_point_columns].sum()

    # 将作为索引的“日期”列重新变回普通列
    aggregated_df = aggregated_df.reset_index()
    print("数据加总完成！")

    # 5. 保存结果到新的Excel文件
    print(f"\n正在将最终结果保存到: {Config.OUTPUT_FILE}")
    try:
        # 确保输出目录存在
        Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
        
        # to_excel 保存数据
        aggregated_df.to_excel(Config.OUTPUT_FILE, index=False, engine='openpyxl')
        
        print("\n--- 全部任务成功完成！ ---")
    except Exception as e:
        print(f"[致命错误] 保存文件时发生错误。")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

--- 开始执行充电负荷数据批量加总任务 ---
发现 14 个Excel文件，准备开始读取...
  -> 正在读取文件: 十堰.xls
  -> 正在读取文件: 咸宁.xls
  -> 正在读取文件: 孝感.xls
  -> 正在读取文件: 宜昌.xls
  -> 正在读取文件: 恩施.xls
  -> 正在读取文件: 武汉.xls
  -> 正在读取文件: 神农架.xls
  -> 正在读取文件: 荆州.xls
  -> 正在读取文件: 荆门.xls
  -> 正在读取文件: 襄阳.xls
  -> 正在读取文件: 鄂州.xls
  -> 正在读取文件: 随州.xls
  -> 正在读取文件: 黄冈.xls
  -> 正在读取文件: 黄石.xls

所有文件读取完毕，正在合并数据...
数据合并完成！
正在进行数据清洗和类型转换...
数据清洗完成！
正在按天对所有地市的负荷进行加总...
数据加总完成！

正在将最终结果保存到: E:\A智网\分地市202508充电负荷数据\湖北省充电负荷数据.xlsx

--- 全部任务成功完成！ ---
